In [ ]:
using PyPlot
using AlfvenDetectors
using DelimitedFiles

In [ ]:
datapath = "/home/vit/vyzkum/alfven/cdb_data/uprobe_data/"

In [ ]:
files = readdir(datapath);
length(files)

In [ ]:
patchsize = 128
outpath = "/home/vit/.julia/environments/v1.1/dev/AlfvenDetectors/experiments/conv/data/"
outfile = joinpath(outpath,"labeled_patches.csv")
labelarray = Array{Any,2}(["shot" "tstart" "fstart" "label"])

In [ ]:
data,header=readdlm(outfile,',',header=true)

### If there are some saved labels already, load them into the label array

In [ ]:
labelarray = readdlm(outfile,',')

### First show the full shot

In [ ]:
shotno = 10000 + 879
file = joinpath(datapath, filter(x->occursin("$shotno", x), files)[1])
# use getsignal so that some of the data is ommited as not being interesting
data = AlfvenDetectors.get_signal(file, AlfvenDetectors.readnormlogupsd; memorysafe=true);
t = AlfvenDetectors.get_signal(file, AlfvenDetectors.readtupsd; memorysafe=true);
f = AlfvenDetectors.readfupsd(file; memorysafe=true);
size(data)
cmap = "inferno"
figure(figsize=(10,5))
title(shotno)
pcolormesh(t,f,data,cmap=cmap)

In [ ]:
mscfile = "/home/vit/vyzkum/alfven/cdb_data/data_sample/Jldata$(shotno).h5"
mscdata = AlfvenDetectors.readmscamp(mscfile, 12; memorysafe=true);
tmsc = AlfvenDetectors.readtcoh(mscfile; memorysafe=true)
fmsc = AlfvenDetectors.readfcoh(mscfile; memorysafe=true)
pcolormesh(tmsc, fmsc, mscdata, cmap=cmap)

### Then cut just the chirp data

In [ ]:
figure(figsize=(10,5))
title(shotno)
ylim = [1000000,2500000]
xlim = [1.075, 1.257]
yinds = ylim[1] .< f .< ylim[2]
xinds = xlim[1] .< t .< xlim[2]
pcolormesh(t[xinds],f[yinds],data[yinds,xinds],cmap=cmap)
xlabel("t [s]")
ylabel("f [Hz]")

In [ ]:
figure(figsize=(10,5))
title(shotno)
xinds = xlim[1] .< tmsc .< xlim[2]
pcolormesh(tmsc[xinds],fmsc,mscdata[:,xinds],cmap=cmap)
xlabel("t [s]")
ylabel("f [Hz]")

### Finally, cut the individual patches

In [ ]:
#fstart = ylim[1]
#fstart = fpatch[end]
#fstart = minimum(f)
fstart = 1000000

#tstart = xlim[1]
tstart = tpatch[end]
#tstart = minimum(t)
tinds = tstart .< t
finds = fstart .< f
tpatch = t[tinds][1:patchsize]
fpatch = f[finds][1:patchsize]
patch = data[finds,tinds][1:patchsize,1:patchsize]
pcolormesh(tpatch, fpatch, patch,cmap=cmap)
label = 1

In [ ]:
labelarray = vcat(labelarray, [shotno tstart fstart label])

In [ ]:
savearray = deepcopy(labelarray)

In [ ]:
writedlm(outfile, savearray, ",")

In [ ]:
readdir(joinpath(dirname(pathof(AlfvenDetectors)), "../experiments/conv/data"))